In [40]:
import logging

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
import llama_index
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index.memory import ChatMemoryBuffer

import oci
import ads

# only to check versions!
import oracledb

from oci_utils import load_oci_config
from ads.llm import GenerativeAIEmbeddings, GenerativeAI
from oracle_vector_db import OracleVectorStore
from oci_baai_reranker import OCIBAAIReranker
from oci_llama_reranker import OCILLamaReranker
from tokenizers import Tokenizer

from config import EMBED_MODEL, RERANKER_ID, TOKENIZER
from config_private import COMPARTMENT_OCID, ENDPOINT

In [41]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")
print(f"llama_index version: {llama_index.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649
llama_index version: 0.9.21


In [42]:
cohere_tokenizer = Tokenizer.from_pretrained(TOKENIZER)

In [43]:
# parameters

# for similarity query
TOP_K = 4
# for Reranker
TOP_N = 2

In [44]:
# Build the entire chain
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

# Embedding model
embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=ads.auth.api_keys(oci_config),
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# Vector Store
v_store = OracleVectorStore(verbose=True)

# reranker
# Reranker is deployed in OCI Data Science as a Model Deployment

baai_reranker = OCIBAAIReranker(
    auth=api_keys_config,
    # OCID of the Model Deployment used
    deployment_id=RERANKER_ID,
    region="eu-frankfurt-1",
)

reranker = OCILLamaReranker(oci_reranker=baai_reranker, top_n=TOP_N)

# LLM
llm_oci = GenerativeAI(
    auth=ads.auth.api_keys(oci_config),
    compartment_id=COMPARTMENT_OCID,
    max_tokens=1024,
    # Optionally you can specify keyword arguments for the OCI client, e.g. service_endpoint.
    client_kwargs={"service_endpoint": ENDPOINT},
)

# Llama-index
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

2024-01-04 13:50:49,069 - INFO - Created OCI reranker client...
2024-01-04 13:50:49,069 - INFO - Region: eu-frankfurt-1...
2024-01-04 13:50:49,070 - INFO - Deployment id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaulxbosgii6yajt2jdsrrvfbequkxt3mepz675uk3ui3q...
2024-01-04 13:50:49,070 - INFO - 


#### To add message_history, create a chat_engine

In [47]:
memory = ChatMemoryBuffer.from_defaults(
    token_limit=2200, tokenizer_fn=cohere_tokenizer.encode
)

chat_engine = index.as_chat_engine(
    chat_mode="context",
    memory=memory,
    verbose=True,
    similarity_top_k=TOP_K,
    node_postprocessors=[reranker],
)

In [48]:
# start a new conversation
chat_engine.reset()

In [49]:
memory.chat_history

[]

In [50]:
response = chat_engine.chat("What is the last release for Oracle database?")

print(response.response)

2024-01-04 13:52:15,441 - INFO - ---> Calling query on DB
2024-01-04 13:52:15,608 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 13:52:15,957 - INFO - Query duration: 0.5 sec.
2024-01-04 13:52:15,959 - INFO - Reranking...


 The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. 

It's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? 


In [51]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. \n\nIt's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? ", additional_kwargs={})]

In [52]:
response = chat_engine.chat("Is it a long term release?")

print(response.response)

2024-01-04 13:52:29,590 - INFO - ---> Calling query on DB
2024-01-04 13:52:29,797 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 13:52:30,161 - INFO - Query duration: 0.6 sec.
2024-01-04 13:52:30,164 - INFO - Reranking...


 Yes, Oracle Database 23c is a long-term support release. 

Oracle Database Releases 19c and 21c were also designated Long Term Support Releases (LTSR) and received updates for extended periods compared to non-LTSRs. However, Oracle Database 23c is the latest LTSR and will remain supported for many years to come, 
but oracle has not yet specified exactly how many years. 

Oracle announces new versions and releases every year, so it is likely that a newer version may come out in the future. Would you like to know more about Oracle Database releases? 


In [53]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. \n\nIt's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? ", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. \n\nOracle Database Releases 19c and 21c were also designated Long Term Support Releases (LTSR) and received updates for extended periods compared to non-LTSRs. However, Oracle Database 23c is the latest LTSR and will remain supported for many years to come, \nbut oracle has not

In [54]:
response = chat_engine.chat("What about new features for JSON?")

print(response.response)

2024-01-04 13:52:48,027 - INFO - ---> Calling query on DB
2024-01-04 13:52:48,215 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 13:52:48,632 - INFO - Query duration: 0.6 sec.
2024-01-04 13:52:48,635 - INFO - Reranking...


 Oracle Database 23c has continued to improve and add features for JSON support. Here are some of the enhanced features:

1. **JSON-Relational Duality Views**: Duality views are a game-changing feature that allows you to store data in a normalized, efficient format in relational tables while providing JSON documents as an interface for applications to access the data. This simplifies the process of working with both relational and document models.

2. **JSON Schema Validation**: While JSON is typically schemaless, providing flexibility, JSON Schema Validation allows you to enforce mandatory fixed structures and typing on your JSON data. This can be useful for ensuring consistency in your data.

3. **Consistent XML and JSON Search Index Syntax**: The syntax for creating and working with XML and JSON search indexes is now consistent, which increases productivity. Additionally, the performance of these search indexes has been improved.

It's important to note that these are just a few of 

In [55]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. \n\nIt's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? ", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. \n\nOracle Database Releases 19c and 21c were also designated Long Term Support Releases (LTSR) and received updates for extended periods compared to non-LTSRs. However, Oracle Database 23c is the latest LTSR and will remain supported for many years to come, \nbut oracle has not

In [56]:
response = chat_engine.chat("Yes give me some more details regarding the n. 1")

print(response.response)

2024-01-04 13:53:07,866 - INFO - ---> Calling query on DB
2024-01-04 13:53:08,064 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 13:53:08,509 - INFO - Query duration: 0.6 sec.
2024-01-04 13:53:08,512 - INFO - Reranking...


 Sure, here are some more details regarding the first feature mentioned, JSON-Relational Duality Views. 

Duality views are a powerful tool for integrating relational and document data models in Oracle Database 23c. They allow you to store data in a structured, relational format while providing a JSON interface for applications to access the same data. This approach offers several advantages.

1. **Data Consistency**: With duality views, you can ensure that your data is consistent across both the relational and document models. Any changes made to the data through one interface will be automatically reflected in the other, eliminating the risk of data inconsistency. 

2. **Ease of Use**: Relational databases are well-established and provide a familiar interface for working with structured data. On the other hand, JSON is widely used for handling unstructured or semi-structured data. Duality views bridge this gap, allowing you to work with JSON data using the same querying and manipulat

In [58]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. \n\nIt's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? ", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. \n\nOracle Database Releases 19c and 21c were also designated Long Term Support Releases (LTSR) and received updates for extended periods compared to non-LTSRs. However, Oracle Database 23c is the latest LTSR and will remain supported for many years to come, \nbut oracle has not

In [67]:
response = chat_engine.chat("What about the symptoms for long covid?")

print(response.response)

2024-01-04 14:30:07,603 - INFO - ---> Calling query on DB
2024-01-04 14:30:07,780 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 14:30:08,129 - INFO - Query duration: 0.5 sec.
2024-01-04 14:30:08,132 - INFO - Reranking...


 Long COVID is an umbrella term for a range of new or ongoing symptoms and medical conditions that can occur four or more weeks after a COVID-19 infection. While long COVID manifests differently in each patient, common symptoms include:

1. Cardiopulmonary symptoms: These are symptoms that affect the heart and lungs, such as shortness of breath, fatigue, chest pain, palpitations, and fever. 

2. Neurocognitive symptoms: These are symptoms that affect the brain and nervous system, such as brain fog, difficulty with memory and concentration, headaches, and dizziness. 

3. Gastrointestinal symptoms: These are symptoms that affect the stomach and intestines, such as nausea, vomiting, diarrhea, constipation, and altered sense of smell or taste.

4. Musculoskeletal symptoms: These are symptoms that affect the muscles, bones, and joints, such as myalgia, arthralgia, and arthritis.

5. Psychiatric symptoms: These are symptoms that affect the mind and emotions, such as depression, anxiety, inso

In [63]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=" The last release for Oracle Database that I could find is Oracle Database 23c, released in October 2023. \n\nIt's important to note that releases are constantly updated and changed, so this may not be the latest information. Would you like to know more about this release? ", additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. \n\nOracle Database Releases 19c and 21c were also designated Long Term Support Releases (LTSR) and received updates for extended periods compared to non-LTSRs. However, Oracle Database 23c is the latest LTSR and will remain supported for many years to come, \nbut oracle has not

In [64]:
memory.reset()

memory.chat_history

[]

In [65]:
response = chat_engine.chat("Tell me more about Respiratory issues in long covid")

print(response.response)

2024-01-04 14:00:08,899 - INFO - ---> Calling query on DB
2024-01-04 14:00:09,157 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-04 14:00:09,472 - INFO - Query duration: 0.6 sec.
2024-01-04 14:00:09,474 - INFO - Reranking...


 Long COVID is an umbrella term for a wide range of new, returning or ongoing health problems people can experience four or more weeks after first being infected with SARS-CoV-2, the virus that causes COVID-19. 

Symptoms of long COVID may also include respiratory issues such as coughing, shortness of breath, and fatigue. Other symptoms include insomnia, changes in smell and/or taste, gastrointestinal symptoms, headaches, anxiety and depression, myalgia and arthralgia, and neurological symptoms. In addition, reproductive symptoms including menstrual disorders and abortive infertility have also been reported. 

The onset of long COVID can be gradual, with symptoms appearing or worsening over time. Long COVID may last for months or years, depending on the individual. In some cases, symptoms may never fully resolve. 

It is important to note that long COVID is a complex and dynamic condition, and research is still ongoing to understand its causes and potential treatments. If you are exper

In [66]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='Tell me more about Respiratory issues in long covid', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Long COVID is an umbrella term for a wide range of new, returning or ongoing health problems people can experience four or more weeks after first being infected with SARS-CoV-2, the virus that causes COVID-19. \n\nSymptoms of long COVID may also include respiratory issues such as coughing, shortness of breath, and fatigue. Other symptoms include insomnia, changes in smell and/or taste, gastrointestinal symptoms, headaches, anxiety and depression, myalgia and arthralgia, and neurological symptoms. In addition, reproductive symptoms including menstrual disorders and abortive infertility have also been reported. \n\nThe onset of long COVID can be gradual, with symptoms appearing or worsening over time. Long COVID may last for months or years, depending on the individual. In some cases, symptoms

In [ ]:
memory.